In [2]:
import numpy as np
import pandas as pd
df = pd.read_excel('dataset.xlsx', sheet_name = 'Smile-IC50-CC50')
df2 = pd.read_excel('dataset.xlsx', sheet_name = 1)
df2.drop(labels = ['molecule', 'Pictures', 'SMILES', 'IC50, mmg/ml', 'CC50-MDCK, mmg/ml', 'SI'], axis = 1, inplace = True)
df = df.join(df2)
df.drop(labels = ['Pictures', 'Title'], axis = 1, inplace = True)
df = df.iloc[:,:-2]
df = df.dropna()
unique_substances = list(df['SMILES'].drop_duplicates())
new_df = None
for subs in unique_substances:
  subdf = df[df['SMILES'] == subs]
  df.loc[df['SMILES'] == subs, 'IC50, mmg/ml'] = np.array([subdf['IC50, mmg/ml'].mean()]*subdf.shape[0]).reshape(-1, 1)
  df.loc[df['SMILES'] == subs, 'CC50-MDCK, mmg/ml'] = np.array([subdf['CC50-MDCK, mmg/ml'].mean()]*subdf.shape[0]).reshape(-1, 1)
  if new_df is None:
    new_df = df[df['SMILES'] == subs].head(1)
  else:
    new_df = pd.concat([new_df, df[df['SMILES'] == subs].head(1)])
new_df.drop(labels = ['SI'], axis = 1, inplace = True)

In [3]:
from rdkit import Chem
from rdkit.Chem import Descriptors
descriptors = {"HeavyAtomCount": Descriptors.HeavyAtomCount,
               "NHOHCount": Descriptors.NHOHCount,
               "NOCount": Descriptors.NOCount,
               "NumHAcceptors": Descriptors.NumHAcceptors,
               "NumHDonors": Descriptors.NumHDonors,
               "NumHeteroatoms": Descriptors.NumHeteroatoms,
               "NumRotatableBonds": Descriptors.NumRotatableBonds,
               "NumValenceElectrons": Descriptors.NumValenceElectrons,
               "NumAromaticRings": Descriptors.NumAromaticRings,
               "NumAliphaticHeterocycles": Descriptors.NumAliphaticHeterocycles,
               "RingCount": Descriptors.RingCount,
               "MW": Descriptors.MolWt,
               "LogP": Descriptors.MolLogP,
               "MR": Descriptors.MolMR,
               "TPSA": Descriptors.TPSA}
for k, f in descriptors.items():
  col = [f(Chem.MolFromSmiles(i)) for i in np.array(new_df['SMILES'])]
  new_df.insert(loc=new_df.shape[1], column=k, value=col)

[15:28:08] Conflicting single bond directions around double bond at index 55.
[15:28:08]   BondStereo set to STEREONONE and single bond directions set to NONE.
[15:28:08] Conflicting single bond directions around double bond at index 55.
[15:28:08]   BondStereo set to STEREONONE and single bond directions set to NONE.
[15:28:09] Conflicting single bond directions around double bond at index 55.
[15:28:09]   BondStereo set to STEREONONE and single bond directions set to NONE.
[15:28:09] Conflicting single bond directions around double bond at index 55.
[15:28:09]   BondStereo set to STEREONONE and single bond directions set to NONE.
[15:28:10] Conflicting single bond directions around double bond at index 55.
[15:28:10]   BondStereo set to STEREONONE and single bond directions set to NONE.
[15:28:10] Conflicting single bond directions around double bond at index 55.
[15:28:10]   BondStereo set to STEREONONE and single bond directions set to NONE.
[15:28:10] Conflicting single bond direc

In [4]:
new_df = new_df.drop(['SMILES'], axis=1)
top_features = [(new_df.iloc[:,i].corr(new_df['IC50, mmg/ml']), new_df.columns[i]) for i in range(2, new_df.shape[1])]
sorted(top_features, key=lambda x: abs(x[0]), reverse=True)[:10]
new_df = new_df[new_df['IC50, mmg/ml'] > new_df['IC50, mmg/ml'].quantile(0.25)]
new_df = new_df[new_df['CC50-MDCK, mmg/ml'] < new_df['CC50-MDCK, mmg/ml'].quantile(0.75)]
top_features = [(new_df.iloc[:,i].corr(new_df['IC50, mmg/ml']), new_df.columns[i]) for i in range(2, new_df.shape[1])]
top_features = sorted(top_features, key=lambda x: abs(x[0]), reverse=True)[:10]

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
x_train, x_test, y_train, y_test = train_test_split(new_df[[v for k, v in top_features]], new_df['IC50, mmg/ml'], test_size=0.3)
params = {'n_estimators':200,
          'max_depth':12,
          'criterion':'squared_error',
          'learning_rate':0.03,
          'min_samples_leaf':16,
          'min_samples_split':16}
model = GradientBoostingRegressor(**params)
model = model.fit(x_train, y_train)
mean_squared_error(model.predict(x_test), y_test)

1143.4902960004342